# Global AI Job Market and Salary Trends 2025: ETL Pipeline

 This notebook outlines the complete Extract, Transform, Load (ETL) process for the Global AI Job Market and Salary Trends dataset. The goal is to prepare a clean, consistent, and analysis-ready dataset for further exploration and dashboarding.


In [12]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [13]:
# Set global display options for pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Define standard paths (adjust if your notebook is in a different directory relative to 'data')
RAW_DATA_PATH = "../data/inputs/raw/"
PROCESSED_DATA_PATH = "../data/inputs/raw/"
FINAL_DATA_PATH = "../data/inputs/cleaned/"

# Create directories if they don't exist
os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)
os.makedirs(FINAL_DATA_PATH, exist_ok=True)

# ETL Step 1: Data Extraction & Combination

## Objective
 Merge the two raw CSV files (ai_job_dataset.csv and ai_job_dataset1.csv) into a single consolidated dataset, while handling differences in columns.

## Datasets
 - **ai_job_dataset.csv** - Part 1 (19 columns, no `salary_local`)
 - **ai_job_dataset1.csv** - Part 2 (20 columns, includes `salary_local`)

## Process
 1. Load both CSV files with pandas.
 2. Align their columns by adding any missing columns (like `salary_local`) with `NaN` values in the first dataset.
 3. Concatenate the datasets into one.
 4. Display initial statistics (`df.describe()`) of the combined data.
 5. Save the combined dataset as `ai_jobs_combined.csv` in the `processed` directory for the next ETL steps.


In [14]:
# Load the dataset
df1 = pd.read_csv(os.path.join(RAW_DATA_PATH, "ai_job_dataset.csv"))
df2 = pd.read_csv(os.path.join(RAW_DATA_PATH, "ai_job_dataset1.csv"))

# Align the columns of the two dataframes
for col in df2.columns:
    if col not in df1.columns:
        df1[col] = pd.NA # Fill missing columns with NA

# Ensure both datasets have same column order
df1 = df1[df2.columns]

# Concatenate the two datasets
combined_df = pd.concat([df1, df2], ignore_index=True)

print(f"Combined dataset created. Total rows: {combined_df.shape[0]}")
print(f"Final columns: {combined_df.columns.tolist()}")

print("\n--- Initial Statistics of Combined Data (df.describe()) ---")
display(combined_df.describe(include='all')) # Include 'all' for mixed types

# Save the combined dataset to the processed directory
combined_df.to_csv(os.path.join(PROCESSED_DATA_PATH, "ai_jobs_combined.csv"), index=False)
print(f"Combined dataset saved to {os.path.join(PROCESSED_DATA_PATH, 'ai_jobs_combined.csv')}")


Combined dataset created. Total rows: 30000
Final columns: ['job_id', 'job_title', 'salary_usd', 'salary_currency', 'salary_local', 'experience_level', 'employment_type', 'company_location', 'company_size', 'employee_residence', 'remote_ratio', 'required_skills', 'education_required', 'years_experience', 'industry', 'posting_date', 'application_deadline', 'job_description_length', 'benefits_score', 'company_name']

--- Initial Statistics of Combined Data (df.describe()) ---


,job_id,job_title,salary_usd,salary_currency,salary_local,experience_level,employment_type,company_location,company_size,employee_residence,remote_ratio,required_skills,education_required,years_experience,industry,posting_date,application_deadline,job_description_length,benefits_score,company_name
count,30000,30000,30000.000000,30000,15000.0,30000,30000,30000,30000,30000,30000.000000,30000,30000,30000.000000,30000,30000,30000,30000.000000,30000.000000,30000
unique,15000,20,NaN,8,14458.0,4,4,20,3,50,NaN,25702,4,NaN,15,486,543,NaN,NaN,16
top,AI00001,Machine Learning Engineer,NaN,USD,131854.0,EX,CT,Switzerland,L,Switzerland,NaN,"Python, TensorFlow, PyTorch",Bachelor,NaN,Consulting,2024-08-25,2024-10-07,NaN,NaN,TechCorp Inc
freq,2,1596,NaN,19410,3.0,7603,7562,1565,10085,1417,NaN,25,7652,NaN,2041,85,87,NaN,NaN,1920
mean,NaN,NaN,118670.451700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.840000,NaN,NaN,6.309433,NaN,NaN,NaN,1502.083667,7.501907,NaN
std,NaN,NaN,62229.977054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.829278,NaN,NaN,5.572413,NaN,NaN,NaN,575.418018,1.447517,NaN
min,NaN,NaN,16621.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,500.000000,5.000000,NaN
25%,NaN,NaN,72575.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,2.000000,NaN,NaN,NaN,1001.000000,6.300000,NaN
50%,NaN,NaN,103206.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.000000,NaN,NaN,5.000000,NaN,NaN,NaN,1512.000000,7.500000,NaN
75%,NaN,NaN,150921.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,NaN,NaN,10.000000,NaN,NaN,NaN,1997.000000,8.800000,NaN


Combined dataset saved to ../data/inputs/raw/ai_jobs_combined.csv


# ETL Step 2: Handling Missing Values

## Objective
 Address missing values in the combined dataset, ensuring data completeness without unnecessary loss of information.

## Focus Areas
 - **`salary_local`**: This column was introduced from `df2` and is `NaN` for all rows originating from `df1`. We will fill these with a meaningful placeholder.
 - **Critical Columns**: Identify and handle missing values in core columns (`job_id`, `job_title`, `salary_usd`). If these are missing, it might indicate corrupted records.

## Process
 1. Load the `ai_jobs_combined.csv` dataset.
 2. Fill `NaN` values in `salary_local` with 'Not Provided'.
 3. Check for `NaN` values in critical columns and report/handle them (e.g., drop rows if truly unrecoverable for core identifiers).


In [15]:
# Load the combined dataset
df = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, "ai_jobs_combined.csv"), dtype={'salary_local': str}) # Load salary_local as string to avoid type issues

print("--- Missing Value Handling ---")
print("Missing values before handling:\n", df.isnull().sum()[df.isnull().sum() > 0])

--- Missing Value Handling ---
Missing values before handling:
 salary_local    15000
dtype: int64


###  Handle `salary_local` missing values 

In [16]:
# Fill NaN values in 'salary_local' with 'Not Provided' to retain rows
df['salary_local'] = df['salary_local'].fillna('Not Provided')
print("\n'salary_local' NaN values filled with 'Not Provided'.")



'salary_local' NaN values filled with 'Not Provided'.


In [17]:
#  Check for and handle critical column missing values 
critical_columns = ['job_id', 'job_title', 'salary_usd']
missing_critical = df[critical_columns].isnull().sum()
if missing_critical.sum() > 0:
    print("\nWARNING: Missing values found in critical columns:")
    print(missing_critical[missing_critical > 0])

    initial_rows = df.shape[0]
    df.dropna(subset=critical_columns, inplace=True)
    rows_dropped = initial_rows - df.shape[0]
    if rows_dropped > 0:
        print(f"{rows_dropped} rows dropped due to missing critical information.")
else:
    print("\nNo missing values found in critical columns.")

print("\nMissing values after handling:\n", df.isnull().sum()[df.isnull().sum() > 0])



No missing values found in critical columns.

Missing values after handling:
 Series([], dtype: int64)


# ETL Step 3: Data Cleaning & Quality

## Objective
 Refine data quality by standardizing formats, removing inconsistencies, and handling outliers.

## Process
 1. **Column Names**: Standardize all column names to lowercase with underscores.
 2. **Text Fields**: Clean and standardize text fields (`job_title`, `company_name`, `company_location`, `employee_residence`, `industry`, `required_skills`) to Title Case and strip extra spaces.
 3. **Date Conversion**: Convert `posting_date` and `application_deadline` to datetime objects.
 4. **Data Types**: Ensure `salary_usd` is numeric.
 5. **Duplicate Removal**: Remove duplicate rows based on `job_id`.
 6. **Outlier Handling (`salary_usd`)**: Remove unrealistic salary values to ensure data quality.


In [18]:
print("\n--- Data Cleaning & Quality ---")

# Standardize Column Names
df.columns = df.columns.str.lower().str.replace(' ', '_')
print("Column names standardized.")
print("New columns: ", df.columns.tolist())

# Clean and Standardize Text Fields
text_cols = ['job_title', 'company_name', 'company_location', 'employee_residence', 'industry', 'required_skills']
for col in text_cols:
    if col in df.columns:
        # Fill NaN with empty string before applying string methods to avoid errors
        df[col] = df[col].fillna('').astype(str).str.strip().str.title()
print("Text fields standardized (stripped, title cased). ")

# Convert Date Columns 
date_cols = ['posting_date', 'application_deadline']
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
print("Date columns converted to datetime objects.")

# Ensure salary_usd is numeric
df['salary_usd'] = pd.to_numeric(df['salary_usd'], errors='coerce')
# After coercing, if any NaNs are created, fill them to maintain data integrity
# A strategy like median or mean could be used, or mark as 'unknown' if not critical for analysis
df['salary_usd'].fillna(df['salary_usd'].median(), inplace=True) # Impute with median for numerical consistency
print("'salary_usd' ensured as numeric, NaNs filled with median.")

# Remove Duplicate Rows based on job_id
initial_rows = df.shape[0]
df.drop_duplicates(subset=['job_id'], inplace=True)
rows_dropped = initial_rows - df.shape[0]
if rows_dropped > 0:
    print(f"{rows_dropped} duplicate rows removed based on 'job_id'.")
else:
    print("No duplicate rows found based on 'job_id'.")

# Outlier Handling for salary_usd (remove unrealistic values) 
salary_min = 1000  # Minimum plausible salary (e.g., to filter out data entry errors or very low part-time gigs if not relevant)
salary_max = 1_000_000 # Maximum plausible salary for general AI roles

initial_rows = df.shape[0]
df = df[(df['salary_usd'] >= salary_min) & (df['salary_usd'] <= salary_max)]
rows_removed_outliers = initial_rows - df.shape[0]
print(f"{rows_removed_outliers} rows removed due to 'salary_usd' outliers (outside ${salary_min}-${salary_max}).")

print("Data cleaning and quality checks completed.")



--- Data Cleaning & Quality ---
Column names standardized.
New columns:  ['job_id', 'job_title', 'salary_usd', 'salary_currency', 'salary_local', 'experience_level', 'employment_type', 'company_location', 'company_size', 'employee_residence', 'remote_ratio', 'required_skills', 'education_required', 'years_experience', 'industry', 'posting_date', 'application_deadline', 'job_description_length', 'benefits_score', 'company_name']
Text fields standardized (stripped, title cased). 
Date columns converted to datetime objects.
'salary_usd' ensured as numeric, NaNs filled with median.
15000 duplicate rows removed based on 'job_id'.
0 rows removed due to 'salary_usd' outliers (outside $1000-$1000000).
Data cleaning and quality checks completed.


# ETL Step 4: Data Transformation & Feature Engineering

## Objective
 Create new, insightful features and standardize existing categorical data to enhance readability and analytical capabilities.

## Process
 1. **`salary_category`**: Categorize `salary_usd` into 'Low', 'Mid', 'High' for readability.
 2. **`remote_status`**: Derive remote work status from `remote_ratio`.
 3. **Time Features**: Extract `posting_year` and `posting_month` from `posting_date`.
 4. **`experience_level` Mapping**: Map abbreviations to full, descriptive names.
 5. **`employment_type` Mapping**: Standardize employment type names.
 6. **`company_size` Mapping**: Standardize company size names and set a categorical order.
 7. **Column Reordering**: Reorder columns for logical presentation.
 8. **Save Final Data**: Save the fully transformed dataset as `ai_jobs_final.csv`.


In [19]:
print("\n--- Data Transformation & Feature Engineering ---")

# Categorize Salary (USD) into 'salary_category'
def categorize_salary(salary):
    if salary < 50000:
        return 'Low'
    elif 50000 <= salary <= 100000:
        return 'Mid'
    else: # salary > 100000
        return 'High'

df['salary_category'] = df['salary_usd'].apply(categorize_salary)
print("'salary_category' created: Low (<$50k), Mid ($50k-$100k), High (>$100k).")
print("Salary Category Distribution:\n", df['salary_category'].value_counts())

# Create 'remote_status' from 'remote_ratio' 
def get_remote_status(ratio):
    if ratio == 100:
        return 'Fully Remote'
    elif ratio == 0:
        return 'On-site'
    else:
        return 'Hybrid'

df['remote_status'] = df['remote_ratio'].apply(get_remote_status)
print("'remote_status' created.")

# Extract Time Features from 'posting_date'
df['posting_year'] = df['posting_date'].dt.year
df['posting_month'] = df['posting_date'].dt.month
print("'posting_year' and 'posting_month' created.")

# Map 'experience_level' abbreviations to full names
print("\nOriginal unique 'experience_level':", df['experience_level'].unique())
experience_level_mapping = {
    'EN': 'Entry-level',
    'MI': 'Mid-level',
    'SE': 'Senior',
    'EX': 'Executive'
}
df['experience_level'] = df['experience_level'].apply(lambda x: experience_level_mapping.get(x, x))
print("Experience levels mapped to full names.")
print("Updated unique 'experience_level':", df['experience_level'].unique())

# Map 'employment_type' for standardization 
print("\nOriginal unique 'employment_type':", df['employment_type'].unique())
employment_type_mapping = {
    'FT': 'Full-time',
    'PT': 'Part-time', # Corrected to lowercase 'part-time' for consistency
    'CT': 'Contract',
    'FL': 'Freelance'
}
df['employment_type'] = df['employment_type'].apply(lambda x: employment_type_mapping.get(x, x))
print("Employment types standardized.")
print("Updated unique 'employment_type':", df['employment_type'].unique())

# Map 'company_size' for standardization and ordering 
print("\nOriginal unique 'company_size':", df['company_size'].unique())
company_size_mapping = {
    'S': 'Small (<50 employees)',
    'M': 'Medium (50-249 employees)',
    'L': 'Large (250+ employees)'
}
df['company_size'] = df['company_size'].apply(lambda x: company_size_mapping.get(x, x))

# Define a categorical order for proper visualization later
company_size_order = [
    'Small (<50 employees)',
    'Medium (50-249 employees)',
    'Large (250+ employees)'
]
# Convert to a categorical type with a defined order
df['company_size'] = pd.Categorical(df['company_size'], categories=company_size_order, ordered=True)
print("Company sizes standardized and ordered.")
print("Updated unique 'company_size':", df['company_size'].unique())

# Reorder Columns for Logical Presentation 
final_columns_order = [
    'job_id', 'job_title', 'company_name', 'company_location', 'employee_residence',
    'industry', 'experience_level', 'employment_type', 'company_size',
    'remote_ratio', 'remote_status',
    'salary_usd', 'salary_currency', 'salary_local', 'salary_category',
    'required_skills', 'education_required', 'years_experience',
    'posting_date', 'posting_year', 'posting_month', 'application_deadline',
    'job_description_length', 'benefits_score'
]

# Ensure all columns in df are in the final_columns_order or append new ones
existing_cols = df.columns.tolist()
final_columns_order = [col for col in final_columns_order if col in existing_cols] # Only include existing columns
for col in existing_cols:
    if col not in final_columns_order:
        final_columns_order.append(col) # Add any new/unexpected columns at the end

df = df[final_columns_order]
print("Columns reordered for final dataset.")


--- Data Transformation & Feature Engineering ---
'salary_category' created: Low (<$50k), Mid ($50k-$100k), High (>$100k).
Salary Category Distribution:
 salary_category
High    7468
Mid     6392
Low     1140
Name: count, dtype: int64
'remote_status' created.
'posting_year' and 'posting_month' created.

Original unique 'experience_level': ['SE' 'EN' 'MI' 'EX']
Experience levels mapped to full names.
Updated unique 'experience_level': ['Senior' 'Entry-level' 'Mid-level' 'Executive']

Original unique 'employment_type': ['CT' 'FL' 'PT' 'FT']
Employment types standardized.
Updated unique 'employment_type': ['Contract' 'Freelance' 'Part-time' 'Full-time']

Original unique 'company_size': ['M' 'L' 'S']
Company sizes standardized and ordered.
Updated unique 'company_size': ['Medium (50-249 employees)', 'Large (250+ employees)', 'Small (<50 employees)']
Categories (3, object): ['Small (<50 employees)' < 'Medium (50-249 employees)' < 'Large (250+ employees)']
Columns reordered for final datase

In [20]:
df.head()

,job_id,job_title,company_name,company_location,employee_residence,industry,experience_level,employment_type,company_size,remote_ratio,remote_status,salary_usd,salary_currency,salary_local,salary_category,required_skills,education_required,years_experience,posting_date,posting_year,posting_month,application_deadline,job_description_length,benefits_score
0,AI00001,Ai Research Scientist,Smart Analytics,China,China,Automotive,Senior,Contract,Medium (50-249 employees),50,Hybrid,90376,USD,Not Provided,Mid,"Tableau, Pytorch, Kubernetes, Linux, Nlp",Bachelor,9,2024-10-18,2024,10,2024-11-07,1076,5.9
1,AI00002,Ai Software Engineer,Techcorp Inc,Canada,Ireland,Media,Entry-level,Contract,Medium (50-249 employees),100,Fully Remote,61895,USD,Not Provided,Mid,"Deep Learning, Aws, Mathematics, Python, Docker",Master,1,2024-11-20,2024,11,2025-01-11,1268,5.2
2,AI00003,Ai Specialist,Autonomous Tech,Switzerland,South Korea,Education,Mid-level,Freelance,Large (250+ employees),0,On-site,152626,USD,Not Provided,High,"Kubernetes, Deep Learning, Java, Hadoop, Nlp",Associate,2,2025-03-18,2025,3,2025-04-07,1974,9.4
3,AI00004,Nlp Engineer,Future Systems,India,India,Consulting,Senior,Freelance,Medium (50-249 employees),50,Hybrid,80215,USD,Not Provided,Mid,"Scala, Sql, Linux, Python",PhD,7,2024-12-23,2024,12,2025-02-24,1345,8.6
4,AI00005,Ai Consultant,Advanced Robotics,France,Singapore,Media,Entry-level,Part-time,Small (<50 employees),100,Fully Remote,54624,EUR,Not Provided,Mid,"Mlops, Java, Tableau, Python",Master,0,2025-04-15,2025,4,2025-06-23,1989,6.6


## Save Cleaned Data
 
 The final step in the ETL process is to save the cleaned and transformed DataFrame. This ensures that the prepared data can be easily accessed for subsequent analysis phases (Exploratory Data Analysis and Machine Learning Modeling) without needing to re-run the entire cleaning script

In [21]:
# Save the final transformed dataset to CSV
df.to_csv(os.path.join(FINAL_DATA_PATH, "ai_jobs_final.csv"), index=False)
print(f"Final transformed dataset saved to {os.path.join(FINAL_DATA_PATH, 'ai_jobs_final.csv')}")

print("\n--- ETL Process Completed ---")
print("Final Dataframe Head:\n", df.head())
print("Final Dataframe Info:\n")


Final transformed dataset saved to ../data/inputs/cleaned/ai_jobs_final.csv

--- ETL Process Completed ---
Final Dataframe Head:
     job_id              job_title       company_name company_location employee_residence    industry experience_level employment_type               company_size  remote_ratio remote_status  salary_usd salary_currency  salary_local salary_category                                  required_skills education_required  years_experience posting_date  posting_year  posting_month application_deadline  job_description_length  benefits_score
0  AI00001  Ai Research Scientist    Smart Analytics            China              China  Automotive           Senior        Contract  Medium (50-249 employees)            50        Hybrid       90376             USD  Not Provided             Mid         Tableau, Pytorch, Kubernetes, Linux, Nlp           Bachelor                 9   2024-10-18          2024             10           2024-11-07                    1076             5.

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 14999
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   job_id                  15000 non-null  object        
 1   job_title               15000 non-null  object        
 2   company_name            15000 non-null  object        
 3   company_location        15000 non-null  object        
 4   employee_residence      15000 non-null  object        
 5   industry                15000 non-null  object        
 6   experience_level        15000 non-null  object        
 7   employment_type         15000 non-null  object        
 8   company_size            15000 non-null  category      
 9   remote_ratio            15000 non-null  int64         
 10  remote_status           15000 non-null  object        
 11  salary_usd              15000 non-null  int64         
 12  salary_currency         15000 non-null  object     